Heroes of Pymoli Data Analysis
1. Most of the players are male, at Least 80% but per capita they only spend slightly more than females. 
2. Most of their players are between the age of 15-24 but no matter the age the spending habits are the same
3. Only 2 of the most popular items are in the top five most lucrative

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
#path of .json file
path_purch_data1='resources/purchase_data.json'
path_purch_data2='resources/purchase_data2.json'

In [4]:
# Open .json read and convert to df
with open(path_purch_data1) as f:
   data = json.load(f)
purch_df = pd.DataFrame(data)

with open(path_purch_data2) as f:
   data2 = json.load(f)
purch2_df = pd.DataFrame(data2)

In [62]:
purch_df = purch_df.append(purch2_df)
purch_df = purch_df.reset_index(drop=True)
#purch_df.head(2)

In [61]:
# List Headers for Reference
#list(purch_df)

In [8]:
# Export to Excel to check answers
writer = pd.ExcelWriter('Heroes_of_Pymoli.xlsx')
purch_df.to_excel(writer,'Data')
writer.save()

In [9]:
player_cnt = len(purch_df.groupby('SN'))
player_cnt_lst = {'Total_Players': [player_cnt]}
player_cnt_df = pd.DataFrame(data=player_cnt_lst)
player_cnt_df

,Total_Players
0,612


In [10]:
#Purchasing Analysis (total)
item_cnt = len(purch_df.groupby('Item ID'))
avg_purch_price = purch_df['Price'].mean()
total_purch_cnt = len(purch_df)
total_revenue = purch_df['Price'].sum()
purch_total = {'Item Count':[item_cnt], 'Avg Purchase Price':[avg_purch_price],
              'Total \nPurchase \nCount':[total_purch_cnt],'Total\nRevenue':[total_revenue]}
purch_total_df = pd.DataFrame(data=purch_total)
purch_total_df

,Avg Purchase Price,Item Count,Total Revenue,Total Purchase Count
0,2.930571,184,2514.43,858


In [11]:
# Isolate gender column and groupby
gender = purch_df[['SN', 'Gender']]
gender_df = gender.groupby('SN').first()

In [12]:
# Gender counts
male_cnt = len(gender_df.loc[gender_df['Gender'] == 'Male'])
female_cnt = len(gender_df.loc[gender_df['Gender'] == 'Female'])
other_cnt = len(gender_df.loc[gender_df['Gender'] == 'Other / Non-Disclosed'])
# Gender Percent
male_pct = male_cnt/player_cnt
female_pct = female_cnt/player_cnt
other_pct = other_cnt/player_cnt

In [13]:
# Create gender demographics df
gender_demographics = {'Male':[male_pct],'Female':[female_pct],'Other':[other_pct]}
gender_demographics_df = pd.DataFrame(data=gender_demographics)
gender_demographics_df['Male'] = pd.Series(["{0:.2f}%".format(val * 100) for val in gender_demographics_df['Male']], index = gender_demographics_df.index)
gender_demographics_df['Female'] = pd.Series(["{0:.2f}%".format(val * 100) for val in gender_demographics_df['Female']], index = gender_demographics_df.index)
gender_demographics_df['Other'] = pd.Series(["{0:.2f}%".format(val * 100) for val in gender_demographics_df['Other']], index = gender_demographics_df.index)
gender_demographics_df

,Female,Male,Other
0,17.65%,80.88%,1.47%


In [14]:
# Gender Purchases
male_purch_df = purch_df.loc[purch_df['Gender'] == 'Male']
female_purch_df = purch_df.loc[purch_df['Gender'] == 'Female']
other_purch_df = purch_df.loc[purch_df['Gender'] == 'Other / Non-Disclosed']

# Gender price avg
male_avg_purch = male_purch_df['Price'].mean()
female_avg_purch = female_purch_df['Price'].mean()
other_avg_purch = other_purch_df['Price'].mean()

# Gender total purchase value
male_total_purch = male_purch_df['Price'].sum()
female_total_purch = female_purch_df['Price'].sum()
other_total_purch = other_purch_df['Price'].sum()

#Gender Normalized purchase Value
male_Ntotal_purch = male_total_purch/male_cnt
female_Ntotal_purch = female_total_purch/female_cnt
other_Ntotal_purch = other_total_purch/other_cnt

In [65]:
#Purchase Analysis (Gender)
purch_analysis_gender = {'Gender':['Male','Female','Other'],
                            'Purchase Count':[male_cnt, female_cnt, other_cnt],
                            'Average Purchase':[male_avg_purch, female_avg_purch, other_avg_purch],
                            'Total Purchase Value':[male_total_purch, female_total_purch, other_total_purch],
                            'Normalized Purchase Value':[male_Ntotal_purch, female_Ntotal_purch, other_Ntotal_purch]}
purch_analysis_gender_df = pd.DataFrame(data=purch_analysis_gender)

In [66]:
purch_analysis_gender_df = purch_analysis_gender_df[['Gender', 'Purchase Count','Average Purchase', 'Total Purchase Value','Normalized Purchase Value']]
purch_analysis_gender_df

,Gender,Purchase Count,Average Purchase,Total Purchase Value,Normalized Purchase Value
0,Male,495,2.944448,2052.28,4.146020
1,Female,108,2.847584,424.29,3.928611
2,Other,9,3.155000,37.86,4.206667


In [82]:
bins = [0, 10, 15, 20, 25, 30, 35, 60]

# Create the names for the four bins
group_names = ['<10', '10-14', '15-19','20-24', '25-29', '30-34','>34']
purch_df['Age_Bin'] = pd.cut(purch_df["Age"], bins, labels=group_names)

In [83]:
#Aggregate Data for bins
purch_age_df = pd.DataFrame(purch_df.groupby(['Age_Bin'], as_index=False)
                        .agg({'Gender':'count', "Price": [np.mean, np.sum]})
                        )
purch_age_df.columns = purch_age_df.columns.droplevel(1)

In [84]:
purch_age2_df = pd.DataFrame(purch_df.groupby(['Age_Bin'], as_index=False)['Price'].sum()
                            .rename(columns={'Price':'Total'}))

merge_age_df = (pd.merge(purch_age_df,purch_age2_df, how='outer',on = 'Age_Bin')
                .rename(columns={"Gender":"Count",'Price':'Mean'})
               )

merge_age_df.columns = ['Age_Bin','Count','Mean', 'Sum', 'Total']
merge_age_df['NTotal'] = merge_age_df['Total']/merge_age_df['Count']
merge_age_df

,Age_Bin,Count,Mean,Sum,Total,NTotal
0,<10,42,2.958571,124.26,124.26,2.958571
1,10-14,86,2.890349,248.57,248.57,2.890349
2,15-19,224,2.848750,638.12,638.12,2.848750
3,20-24,371,2.974259,1103.45,1103.45,2.974259
4,25-29,84,2.873333,241.36,241.36,2.873333
5,30-34,72,2.933333,211.20,211.20,2.933333
6,>34,57,3.080175,175.57,175.57,3.080175


In [80]:
purch_top_df = pd.DataFrame(purch_df.groupby(['SN'], as_index=False)
                           .agg({'Item Name':'count', 'Price':[np.mean,np.sum]}))
purch_top_df.columns = purch_top_df.columns.droplevel(1)
purch_top_df.columns = ['Player','Count', 'Mean', 'Sum']

purch_top_df.sort_values('Sum',ascending=False).head(5)


,Player,Count,Mean,Sum
22,Aerithllora36,5,3.950,19.75
574,Undirrala66,5,3.412,17.06
503,Sondim43,5,3.332,16.66
515,Sundaky74,4,3.705,14.82
360,Lisosiast26,4,3.475,13.90


In [59]:
purch_topitem_df = pd.DataFrame(purch_df.groupby(['Item Name'], as_index=False)
                           .agg({'SN':'count', 'Price':[np.mean,np.sum]}))
purch_topitem_df.columns = purch_topitem_df.columns.droplevel(1)
purch_topitem_df.columns = ['Player','Count', 'Mean', 'Sum']
purch_topitem_df = purch_topitem_df.sort_values('Count',ascending=False).head(5)
purch_topitem_df.reset_index(level=0, inplace=True)
purch_topitem_df.rename(columns={'index':'Item ID'})

,Item ID,Player,Count,Mean,Sum
0,56,Final Critic,14,2.757143,38.60
1,8,Arcane Gem,12,2.445000,29.34
2,138,Stormcaller,12,3.349167,40.19
3,11,"Betrayal, Whisper of Grieving Widows",11,2.350000,25.85
4,156,Trickster,10,2.322000,23.22


In [60]:
#Most profitable item
purch_topitem_df = pd.DataFrame(purch_df.groupby(['Item Name'], as_index=False)
                           .agg({'SN':'count', 'Price':[np.mean,np.sum]}))
purch_topitem_df.columns = purch_topitem_df.columns.droplevel(1)
purch_topitem_df.columns = ['Player','Count', 'Mean', 'Sum']
purch_topitem_df = purch_topitem_df.sort_values('Sum',ascending=False).head(5)
purch_topitem_df.reset_index(level=0, inplace=True)
purch_topitem_df.rename(columns={'index':'Item ID'})

,Item ID,Player,Count,Mean,Sum
0,138,Stormcaller,12,3.349167,40.19
1,56,Final Critic,14,2.757143,38.60
2,113,Retribution Axe,9,4.140000,37.26
3,135,"Splitter, Foe Of Subtlety",9,3.670000,33.03
4,133,Spectral Diamond Doomblade,7,4.250000,29.75
